# 🚀 TinyNN + NeuroATM: Расширенный тест

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class MiniTinyNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(2, 2)
    def forward(self, x):
        return self.fc(x)

In [ ]:
class SimpleATM:
    def __init__(self):
        self.memory = []
    def act(self, context):
        if not self.memory:
            return np.random.randint(2)
        sims = [np.dot(context, mem[0]) for mem in self.memory]
        return self.memory[np.argmax(sims)][1]
    def update(self, context, response, success):
        for i, (ctx, resp, score) in enumerate(self.memory):
            if np.allclose(ctx, context, atol=0.1) and resp == response:
                self.memory[i] = (ctx, resp, score + success)
                return
        self.memory.append((context, response, success))

In [ ]:
def generate_data(seed=1):
    np.random.seed(seed)
    X = np.random.uniform(-1, 1, (100, 2))
    Y = np.array([int(x[0] * x[1] > 0) for x in X])
    return X, Y

X_train, Y_train = generate_data(seed=1)
X_test, Y_test = generate_data(seed=42)

In [ ]:
torch.manual_seed(1)
model = MiniTinyNN()
opt = torch.optim.SGD(model.parameters(), lr=0.05)
loss_fn = nn.CrossEntropyLoss()
for _ in range(200):  # увеличено число эпох
    inp = torch.tensor(X_train, dtype=torch.float32)
    lbl = torch.tensor(Y_train, dtype=torch.long)
    out = model(inp)
    loss = loss_fn(out, lbl)
    opt.zero_grad()
    loss.backward()
    opt.step()

In [ ]:
atm = SimpleATM()
success_train = []
for i in range(len(X_train)):
    input_vec = torch.tensor(X_train[i], dtype=torch.float32)
    with torch.no_grad():
        context = model(input_vec).numpy()
    action = atm.act(context)
    reward = 1 if action == Y_train[i] else -1
    atm.update(context, action, reward)
    success_train.append(success_train[-1] + reward if success_train else reward)

In [ ]:
success_test = []
for i in range(len(X_test)):
    input_vec = torch.tensor(X_test[i], dtype=torch.float32)
    with torch.no_grad():
        context = model(input_vec).numpy()
    action = atm.act(context)
    reward = 1 if action == Y_test[i] else -1
    success_test.append(success_test[-1] + reward if success_test else reward)

In [ ]:
def run_random_agent(Y):
    random_success_log = []
    for true_label in Y:
        action = np.random.randint(2)
        reward = 1 if action == true_label else -1
        if random_success_log:
            random_success_log.append(random_success_log[-1] + reward)
        else:
            random_success_log.append(reward)
    return random_success_log

random_train = [run_random_agent(Y_train) for _ in range(5)]
random_test = [run_random_agent(Y_test) for _ in range(5)]
avg_rand_train = np.mean(random_train, axis=0)
avg_rand_test = np.mean(random_test, axis=0)

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(success_train, label='Обучение: TinyNN+ATM')
plt.plot(avg_rand_train, label='Рандом (обучение)', linestyle='--')
plt.title('На обучающих данных')
plt.xlabel('Итерации')
plt.ylabel('Накопленный успех')
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(success_test, label='Тест: TinyNN+ATM')
plt.plot(avg_rand_test, label='Рандом (тест)', linestyle='--')
plt.title('На новых тестовых данных')
plt.xlabel('Итерации')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()